In [11]:
import pandas as pd 
import numpy as np
import scipy
from scipy import stats
import matplotlib.pyplot as plt

In [6]:
y = ds['SLIV']
x1 = ds['TOTALINC']
x2 = ds['CONSUM']
x = [x1, x2]
f = lambda x, a, b, c: a + b*x[0] + c*x[1]

In [12]:
df = pd.ExcelFile('house01.xls')
ds = df.parse('Дані')
ds = ds[['COD_OBL','SLIV','TOTALINC','CONSUM']]
ds = ds.loc[ds['COD_OBL'] == 'Khark']
print(ds)

     COD_OBL  SLIV   TOTALINC     CONSUM
2814   Khark  43.4   6537.730   5445.395
2815   Khark  27.4   5010.000   8748.480
2816   Khark  43.4   9062.305  10071.220
2817   Khark  42.3   7476.710   7420.655
2818   Khark  18.2   5169.540   7491.460
...      ...   ...        ...        ...
3258   Khark  50.0  12271.540   6336.600
3259   Khark  38.0   5054.980   2287.520
3260   Khark  75.0   5405.025   1717.555
3261   Khark  55.0   4426.905   2244.430
3262   Khark  40.0   8741.850   4486.530

[449 rows x 4 columns]


# 

In [13]:
dx1 = x1 - x1.mean() 
dy = y - y.mean()
covariance = (dx1 * dy).sum() / (dx1.count() - 1)
x1_hat = x1.mean()
n = x1.count()
var_x = sum((x1 - x1_hat) ** 2) / n
std_dev_x = np.sqrt(var_x)
y_hat = y.mean()
var_y = sum((y - y_hat) ** 2) / n
std_dev_y = np.sqrt(var_y)

k_cor = covariance/(std_dev_x * std_dev_y)
print('Correlation y, x1 = ',k_cor)

Correlation y, x1 =  0.2790452743536842


In [15]:
t_stat=k_cor * np.sqrt(( x1.count() - 2)/ 1 - k_cor ** 2)
p = 2 * stats.t.sf(t_stat, (x1.count() - 2))

In [16]:
dx2 = x2 - x2.mean() 
covariance=(dx2 * dy).sum() / (dx2.count() - 1)
x2_hat = x2.mean()
var_x=sum((x2 - x2_hat) ** 2) / n
std_dev_x = np.sqrt(var_x)
k_cor=covariance/(std_dev_x * std_dev_y)
print('Correlation y, x2 = ',k_cor)

Correlation y, x2 =  0.1488401795146879


In [17]:
df = x2.count() - 2
t_stat = k_cor * np.sqrt(df / 1 - k_cor ** 2)
p = 2 * stats.t.sf(t_stat, df)

Коеф  для рівняння прогнозування.

In [18]:
X1=x1.to_numpy()
X2=x2.to_numpy()
Y=y.to_numpy()
list1=[1]*len(X1)
X = np.array((X1, X2,list1))
X = X.T
w = np.linalg.solve(np.dot(X.T, X), np.dot(X.T, Y))
Yhat = np.dot(X, w)
d1 = Y - Yhat
d2 = Y - Y.mean()
r2 = 1 - d1.dot(d1) / d2.dot(d2)
fstat=r2/(1-r2)*((len(Y)-2-1)/2)
ftest=1-stats.f.cdf(fstat,2,len(Y)-2-1)

# Рівняння для прогнозування

In [19]:
print('SLIV =  %.4f*TOTALINC'%w[0],'+%.4f*CONSUM'%w[1],'+%.4f'%w[2])

SLIV =  0.0013*TOTALINC +-0.0002*CONSUM +29.0017


In [16]:
popt, pcov = scipy.optimize.curve_fit(f, x, y)
print(popt)

[ 2.90016795e+01  1.27493464e-03 -2.47454135e-04]


In [18]:
print((((y - f(x, popt[0], popt[1], popt[2]))**2).mean())**(1/2))

13.41167424017873
